<a href="https://colab.research.google.com/github/corneliarsm/Covid_Xray_Detection/blob/master/Riset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To run this colab, press the "Runtime" button in the menu tab and then press the "Run all" button.

# Recognize Rontgen using Transfer Learning

In [0]:
! pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [0]:
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from pathlib import Path
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

tf.__version__

In [0]:
#Download from Kaggle
! kaggle datasets download -d sapal6/waste-classification-data-v2 --force

In [0]:
#Unzip
import zipfile

zipfile = zipfile.ZipFile("/content/waste-classification-data-v2.zip", 'r')
zipfile.extractall("/content/download")
zipfile.close()

In [0]:
!mv /content/download/DATASET/ /content/DATASET

In [0]:
TRAINING_DIR = "/content/DATASET/TRAIN/"
VALIDATION_DIR = "/content/DATASET/TEST/"

## Setup Input Pipeline

Use `ImageDataGenerator` to rescale the images.

Create the train generator and specify where the train dataset directory, image size, batch size.

Create the validation generator with similar approach as the train generator with the flow_from_directory() method.

In [0]:
#ImageData Generator
batch_size = 128
image_size=224
training_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      horizontal_flip=True,
      shear_range = 0.2,
      fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(
      rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(image_size,image_size),
	class_mode='categorical',
  batch_size=batch_size
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(image_size,image_size),
	class_mode='categorical',
  batch_size=batch_size
)

In [0]:
#Get Size train_generator and validation_generator
train_num = train_generator.samples
valid_num = validation_generator.samples

print(train_num)
print(valid_num)

In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

Save the labels in a file which will be downloaded later with TFLITE file

In [0]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels_riset3.txt', 'w') as f:
  f.write(labels)

In [0]:
!cat labels_riset1.txt

## Create the base model from the pre-trained convnets

Create the base model from the **MobileNet V2** model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the `include_top=False` argument, we load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [0]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

## Feature extraction
You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.

In [0]:
base_model.trainable = False

### Add a classification head

In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(3, activation='softmax')
])

### Compile the model

You must compile the model before training it.  Since there are multiple classes, use a categorical cross-entropy loss.

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Train the model

<!-- TODO(markdaoust): delete steps_per_epoch in TensorFlow r1.14/r2.0 -->

In [0]:
epochs = 5

history = model.fit(train_generator, 
                    epochs=epochs, 
                    steps_per_epoch=train_num//batch_size, 
                    validation_data = validation_generator,
                    validation_steps=valid_num//batch_size)

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor. 

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tuning
In our feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were **not** updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic features maps to features associated specifically to our dataset.

### Un-freeze the top layers of the model


All you need to do is unfreeze the `base_model` and set the bottom layers be un-trainable. Then, recompile the model (necessary for these changes to take effect), and resume training.

In [0]:
base_model.trainable = True

In [0]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 155

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Compile the model

Compile the model using a much lower training rate.

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Continue Train the model

In [0]:
history_fine = model.fit(train_generator, 
                    epochs=5, 
                    steps_per_epoch=train_num//batch_size, 
                    validation_data = validation_generator,
                    validation_steps=valid_num//batch_size)

## Convert to TFLite

Saved the model using `tf.saved_model.save` and then convert the saved model to a tf lite compatible format.

In [0]:
saved_model_dir = 'save/fine_tuning_riset1'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model_riset1.tflite', 'wb') as f:
  f.write(tflite_model)

Download the converted model and labels

In [0]:
#from google.colab import files

#files.download('model_riset1.tflite')
#files.download('labels_riset1.txt')

Let's take a look at the learning curves of the training and validation accuracy/loss, when fine tuning the last few layers of the MobileNet V2 base model and training the classifier on top of it. The validation loss is much higher than the training loss, so you may get some overfitting.

You may also get some overfitting as the new training set is relatively small and similar to the original MobileNet V2 datasets.


In [0]:
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
from tensorflow.keras.preprocessing import image

path = "/Users/mac-eko/RisetDeepLearning/Riset Sampah Recycle-Organik/Dataset-organic-recycle/TEST/R/R_10421.jpg"
img = image.load_img(path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=5)
  
  #print(fn)
print(classes)
max_index_col = np.argmax(classes, axis=1)

In [0]:
# Preparing test data
PATH_TEST=Path("/Users/mac-eko/RisetDeepLearning/Riset Sampah Recycle-Organik/Dataset-organic-recycle/TEST")
organic_dir = PATH_TEST / 'O'
recycle_dir = PATH_TEST / 'R'

organic_waste = organic_dir.glob('*.jpg')
recycle_waste = recycle_dir.glob('*.jpg')

test_data = []
test_labels = []

for img in organic_waste:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    label = to_categorical(0, num_classes=2)
    test_data.append(img)
    test_labels.append(label)
                      
#for img in recycle_waste:
#    img = cv2.imread(str(img))
#    img = cv2.resize(img, (224,224))
#    if img.shape[2] ==1:
#        img = np.dstack([img, img, img])
#    else:
#        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#    img = img.astype(np.float32)/255.
#    label = to_categorical(1, num_classes=2)
#    test_data.append(img)
#    test_labels.append(label)
    

test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Total number of test examples: ", test_data.shape)
print("Total number of labels:", test_labels.shape)

In [0]:
# Get predictions
preds = model.predict(test_data, batch_size=16)
preds = np.argmax(preds, axis=-1)

# Original labels
orig_test_labels = np.argmax(test_labels, axis=-1)

print(orig_test_labels.shape)
print(preds.shape)

In [0]:
# Get the confusion matrix
cm  = confusion_matrix(orig_test_labels, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.xticks(range(2), ['Organic', 'Recycle'], fontsize=16)
plt.yticks(range(2), ['Organic', 'Recycle'], fontsize=16)
plt.show()

## Summary:

* **Using a pre-trained model for feature extraction**:  When working with a small dataset, it is common to take advantage of features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully-connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier get updated during training.
In this case, the convolutional base extracted all the features associated with each image and you just trained a classifier that determines the image class given that set of extracted features.

* **Fine-tuning a pre-trained model**: To further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning.
In this case, you tuned your weights such that your model learned high-level features specific to the dataset. This technique is usually recommended when the training dataset is large and very similar to the orginial dataset that the pre-trained model was trained on.
